In [1]:
!pip install hyperas

In [2]:
import h5py
from os.path import join,exists
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.optimizers import RMSprop
from hyperas.distributions import choice
from hyperas import optim
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from random import randint
from keras import backend as K
import os
import numpy as np
from sklearn.utils import shuffle
from hyperopt import Trials, STATUS_OK, tpe
import tensorflow as tf
K.set_image_data_format('channels_first')

In [3]:
os.chdir('/content/drive/My Drive/Colab Notebooks/synopsys-2022/models/regression')

In [21]:
from keras.layers.normalization.batch_normalization import BatchNormalization
def create_model(X_train, Y_train, X_test, Y_test):
  W_maxnorm = 3
  DROPOUT = {{choice([0.3,0.5,0.7])}}

  model = Sequential()
  model.add(Conv2D(64, (1, 5), padding='same', input_shape=(20, 1, 20),activation='relu',kernel_constraint=maxnorm(W_maxnorm)))
  model.add(MaxPool2D(pool_size=(1, 2),strides=(1,2)))
  model.add(Flatten())



  model.add(Dense(32,activation='relu',kernel_constraint=maxnorm(W_maxnorm)))
  model.add(BatchNormalization())

  model.add(Dropout(DROPOUT))
  model.add(Dense(16,activation='relu',kernel_constraint=maxnorm(W_maxnorm)))
  model.add(BatchNormalization())
  model.add(Dropout(DROPOUT))
  model.add(Dense(1))

  myoptimizer = RMSprop(lr={{choice([0.01,0.001,0.0001,1e-5,1e-1])}}, rho=0.9, epsilon=1e-06)
  mylossfunc='mean_squared_error'
  model.compile(loss=mylossfunc, optimizer=myoptimizer)

  result = model.fit(X_train, Y_train, batch_size=100, epochs=5,validation_split=0.1,verbose=False)

  val_loss = np.amax(result.history['val_loss'])

  return {'loss': val_loss, 'status': STATUS_OK, 'model': model}

In [15]:
def train_data():
  data_train = h5py.File('./data/train.h5.batch1','r')
  X_train = np.array(data_train['data'])
  Y_train = np.array(data_train['label'])
  # X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train)

  data_test = h5py.File('./data/test_new.h5.batch1', 'r')
  X_test = np.array(data_test['data'])
  Y_test = np.array(data_test['label'])
  return X_train, Y_train, X_test, Y_test

In [25]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=train_data,
                                          algo=tpe.suggest,
                                          max_evals=1,
                                          trials=Trials(),
                                          notebook_name='seq_64x1_16',
                                          verbose=False)

  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)



100%|██████████| 1/1 [00:12<00:00, 12.61s/it, best loss: 0.3150993883609772]


In [26]:
X_train, Y_train, X_test, Y_test = train_data()
best_model.fit(X_train, Y_train, batch_size=100, epochs=40,validation_split=0.2,verbose=True)

Epoch 1/40
508/508 [==============================] - 2s 5ms/step - loss: 0.3172 - val_loss: 0.4172
Epoch 2/40
508/508 [==============================] - 2s 5ms/step - loss: 0.3043 - val_loss: 0.7223
Epoch 3/40
508/508 [==============================] - 2s 4ms/step - loss: 0.2981 - val_loss: 0.6188
Epoch 4/40
508/508 [==============================] - 2s 4ms/step - loss: 0.2898 - val_loss: 0.5463
Epoch 5/40
508/508 [==============================] - 2s 4ms/step - loss: 0.2859 - val_loss: 0.8327
Epoch 6/40
508/508 [==============================] - 2s 4ms/step - loss: 0.2804 - val_loss: 0.6959
Epoch 7/40
508/508 [==============================] - 2s 4ms/step - loss: 0.2762 - val_loss: 0.8387
Epoch 8/40
508/508 [==============================] - 2s 4ms/step - loss: 0.2729 - val_loss: 1.1165
Epoch 9/40
508/508 [==============================] - 2s 4ms/step - loss: 0.2705 - val_loss: 1.2163
Epoch 10/40
508/508 [==============================] - 2s 4ms/step - loss: 0.2685 - val_loss: 1.2023

KeyboardInterrupt: ignored